In [1]:
import numpy as np
import pandas as pd


from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
import seaborn as sns

from psql_functions import execQuery
from miss_data import add_missing_dates, add_missing_counts
from make_plots import dict_error_plot_epsilons, dict_error_plot_n

import warnings
warnings.filterwarnings('ignore')

import sys
import os
import re
owd = os.getcwd()

In [2]:
param_dic = {
    "host"      : "localhost",
    "database"  : "bachelorBesoeg2014",
    "user"      : "postgres",
    "password"  : "password",
    "port"      : "5432"
}

query = """select time_ from _775147;"""
result = execQuery(param_dic, query)
dates = [(date[0]) for date in result]

query = """select count_ from _775147;"""
result = execQuery(param_dic, query)

counts = [(count[0]) for count in result]

all_dates = add_missing_dates(dates)
all_counts =  add_missing_counts(counts, dates, all_dates)

n_data_structures = 50


Executed query and closed connection.
Executed query and closed connection.


In [3]:
onlydirs = [f for f in os.listdir(os.getcwd()) if os.path.isdir(os.path.join(os.getcwd(), f)) and f[0] != '.']

In [4]:
onlydirs

['Data', 'plots', 'range_queries', 'results', '__pycache__']

In [5]:
def make_lookup_error_dict_e_n(epsilons, n):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)

    i = 0
    for e_key in epsilon_dict:
        epsilon_dict[e_key] = n_dicts[i]
        i = i + 1
    return epsilon_dict

def make_query_lookup_dict_n_r(n, rs):
    n_dict = dict.fromkeys(n)
    r_dicts = []
    for i in range(0,len(n)):
        #print(rs[i])
        r_dict = dict.fromkeys(rs[i])
        r_dicts.append(r_dict)

    for idx, n_key in enumerate(n_dict):
            n_dict[n_key] = r_dicts[idx]
    
    return n_dict

def fill_up_query_lookup_dict_n_r(n, rs, files):
    dict_ = make_query_lookup_dict_n_r(n, rs)
    for idx, f in enumerate(files):
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_r = re.search(r'r=\d+', f).group(0) 
        here_r = here_r.split("r=")[1]
        
        querries = load_range_queries_n_split(f, n_data_structures)
        dict_[int(here_n)][int(here_r)] = querries.copy()
    return dict_

In [6]:
def make_lookup_error_dict(epsilons, n, rs):
    epsilon_dict = dict.fromkeys(epsilons)
    n_dicts = []
    for i in range(0,len(epsilons)):
        n_dict = dict.fromkeys(n)
        n_dicts.append(n_dict)
    r_dicts = []
    for i in range(0,len(n)*len(epsilons)):
        r_dict = dict.fromkeys(rs[i%len(n)])
        r_dicts.append(r_dict)
    i = 0
    j = 0
    for e_key in epsilon_dict:
        epsilon_dict[e_key] = n_dicts[i]
        n_dict = epsilon_dict[e_key]
        for n_key in n_dict:
            #print(j)
            n_dict[n_key] = r_dicts[j]
            j = j + 1
        i = i + 1
    return epsilon_dict


def fill_up_lookup_error_dict(epsilons, n, degrees, files):
    dict_ = make_lookup_error_dict(epsilons, n, degrees)
    for idx, f in enumerate(files):
        e = re.search(r'e=[-+]?[0-9]*\.?[0-9]*', f).group(0)
        e = e.split("e=")[1]
        here_n = re.search(r'N=\d+', f).group(0)
        here_n = here_n.split("N=")[1]
        here_r = re.search(r'r=\d+', f).group(0) 
        here_r = here_r.split("r=")[1]

        dict_[float(e)][int(here_n)][int(here_r)] = np.loadtxt(f, delimiter=',')
    return dict_

def find_error_different_queries(est, cor):
    erorrs = est - cor
    max_error = np.amax(erorrs)
    min_error = np.amin(erorrs)
    rmse = mean_squared_error(cor, est, squared = False)
    error_dict = {'rmse': rmse, 'max': max_error, 'min': min_error, 'abs': np.mean(erorrs)}
    return error_dict

In [7]:
epsilons = np.array([2, 1.4, 1.2, 1, 0.8, 0.6, 0.5, 0.3])
n = np.array([32,128,256,512,1024,2048])
flat_r_32 =  np.array([2, 4, 8, 12, 16, 20, 24])
flat_r_128 =  np.array([20, 40, 50, 60, 70, 80, 90])
flat_r_256 =  np.array([40, 60, 80, 100, 140, 200, 220])
flat_r_512 =  np.array([100, 150, 200, 250, 300, 400, 450])
flat_r_1024 =  np.array([200, 300, 400, 500, 600, 800, 900])
flat_r_2048 =  np.array([600, 800, 1000, 1250, 1500, 1700, 1800])

rs = [flat_r_32, flat_r_128, flat_r_256, flat_r_512, flat_r_1024, flat_r_2048]

In [8]:
len(epsilons)*len(n)*len(rs[0])

336

# Loading the data 

In [9]:
#C:\Users\Jonas Friis\Documents\Universitet\Bachelors-Thesis\src\results\sample_querys\local_hh
os.chdir(os.getcwd()+'/'+'results/sample_querys/flat_varying_r/')
os.getcwd()

'C:\\Users\\Jonas Friis\\Documents\\Universitet\\Bachelors-Thesis\\src\\results\\sample_querys\\flat_varying_r'

In [10]:
files = [f for f in os.listdir('.')]
files_cor = files[:len(files)//2]
files_est = files[len(files)//2:]
print(files_cor)
print(len(files_cor))

print(files_est)
print(len(files_est))
"""
#Where flat beats hh
hh_files_cor_flat = hh_files_cor[:144]
#Where hh beats flat
hh_files_cor_hh = hh_files_cor[144+1:]
#print(hh_files_cor_flat)
#print(hh_files_cor_hh)
hh_files_est = hh_files[len(hh_files)//2:]
hh_files_est_flat = hh_files_est[:144]
hh_files_est_hh = hh_files_est[144+1:]
#print(hh_files_est_flat)
#print(hh_files_est_hh)
"""

['cor_e=0.3_N=1024_r=200.csv', 'cor_e=0.3_N=1024_r=300.csv', 'cor_e=0.3_N=1024_r=400.csv', 'cor_e=0.3_N=1024_r=500.csv', 'cor_e=0.3_N=1024_r=600.csv', 'cor_e=0.3_N=1024_r=800.csv', 'cor_e=0.3_N=1024_r=900.csv', 'cor_e=0.3_N=128_r=20.csv', 'cor_e=0.3_N=128_r=40.csv', 'cor_e=0.3_N=128_r=50.csv', 'cor_e=0.3_N=128_r=60.csv', 'cor_e=0.3_N=128_r=70.csv', 'cor_e=0.3_N=128_r=80.csv', 'cor_e=0.3_N=128_r=90.csv', 'cor_e=0.3_N=2048_r=1000.csv', 'cor_e=0.3_N=2048_r=1250.csv', 'cor_e=0.3_N=2048_r=1500.csv', 'cor_e=0.3_N=2048_r=1700.csv', 'cor_e=0.3_N=2048_r=1800.csv', 'cor_e=0.3_N=2048_r=600.csv', 'cor_e=0.3_N=2048_r=800.csv', 'cor_e=0.3_N=2048_r=900.csv', 'cor_e=0.3_N=256_r=100.csv', 'cor_e=0.3_N=256_r=140.csv', 'cor_e=0.3_N=256_r=200.csv', 'cor_e=0.3_N=256_r=220.csv', 'cor_e=0.3_N=256_r=40.csv', 'cor_e=0.3_N=256_r=60.csv', 'cor_e=0.3_N=256_r=80.csv', 'cor_e=0.3_N=32_r=12.csv', 'cor_e=0.3_N=32_r=16.csv', 'cor_e=0.3_N=32_r=2.csv', 'cor_e=0.3_N=32_r=20.csv', 'cor_e=0.3_N=32_r=24.csv', 'cor_e=0.3_N=3

'\n#Where flat beats hh\nhh_files_cor_flat = hh_files_cor[:144]\n#Where hh beats flat\nhh_files_cor_hh = hh_files_cor[144+1:]\n#print(hh_files_cor_flat)\n#print(hh_files_cor_hh)\nhh_files_est = hh_files[len(hh_files)//2:]\nhh_files_est_flat = hh_files_est[:144]\nhh_files_est_hh = hh_files_est[144+1:]\n#print(hh_files_est_flat)\n#print(hh_files_est_hh)\n'

In [11]:
cor_dict = fill_up_lookup_error_dict(epsilons, n, rs, files_cor)
est_dict = fill_up_lookup_error_dict(epsilons, n, rs, files_est)


ValueError: could not convert string to float: '2014-02-06'

In [ ]:
cor_dict

In [ ]:
est_dict

In [ ]:
error_dict = make_lookup_error_dict(epsilons, n, rs)

for e in epsilons:
    for N in n:
        #r_values = error_dict[e][N]
        for r in error_dict[e][N]:
            error_dict[e][N][r] = find_error_different_queries(est_dict[e][N][r], cor_dict[e][N][r])

In [ ]:
def get_answers_for_r(n_r, step_size, all_answers):
    """
    Setup of the datastructere
    Parameters:
    n_r (int): Which index/number of the r values
    step_size (int): How many r values there is 
    all_answers (Array): All answers for all the differnt r values
    Returns:
    The answers for this specific r
    """
    return all_answers[n_r::step_size]

In [ ]:
dict_plot_rmse_dict = make_lookup_error_dict_e_n(epsilons, n)
for e in epsilons:
    for N in n:
        rmse = []
        for r_keys in error_dict[e][N]: 
            rmse.append(error_dict[e][N][r_keys]['rmse'])
        
        dict_plot_rmse_dict[e][N] = rmse

In [ ]:
dict_plot_rmse_dict

In [ ]:
def plot(xs, ys, N, epsilon, plot_name):

    plt.scatter(xs, ys)
    #plt.hlines(np.mean(answears128),line_range[0],line_range[1])
    #plt.hlines(np.mean(correct_answear),line_range[0],line_range[1])
    #plt.legend()
    plt.xlabel('Length of query')
    plt.ylabel('RMSE')
    plt.title(f'Flat OLH with N = {N} and \u03B5 = {epsilon}')
    #plt.savefig(plot_name,bbox_inches='tight')
    plt.show()

In [ ]:
for e in epsilons:
    for N in n:
        xs = list(error_dict[e][N].keys())
        print(xs)
        ys = dict_plot_rmse_dict[e][N]
        print(ys)
        plot(xs, ys, N, e, 'e')
